<a href="https://colab.research.google.com/github/ATLAS097/Data-Engineering-CA2/blob/Ben/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning
---
### Date: 21 July 2025
### Class: DAAA/FT/2B/22
### Group-5
### Group Member: Kaung Myat San, Swam Htet Aung, Ethan Ng Wen Xian
---

## Loading Modules

In [1]:
!pip install python-docx

import pandas as pd
from docx import Document

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.7 MB/s eta 0:00:00


## Loading Data
---

In [2]:
# read appointment.xlsx from a URL (must use the raw file link and specify engine)
appointment = pd.read_excel(
	"https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/appointment.xlsx",
	engine="openpyxl"
)

In [20]:
!wget https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/facility.docx -O facility.docx
document = Document("facility.docx")

table = document.tables[0]

data = []
for row in table.rows:
    data.append([cell.text for cell in row.cells])

# Convert to pandas DataFrame
facility = pd.DataFrame(data)

# Set the first row as header and remove it from data
facility.columns = facility.iloc[0]
facility = facility[1:].reset_index(drop=True)

display(facility.head())

--2025-07-24 11:27:22--  https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/facility.docx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34082 (33K) [application/octet-stream]
Saving to: ‘facility.docx’

facility.docx       100%[===================>]  33.28K  --.-KB/s    in 0.001s  

2025-07-24 11:27:22 (27.2 MB/s) - ‘facility.docx’ saved [34082/34082]



,Department,BuldingID,BuildingName,RoomID,Purpose
0,Emergency,B0001,Emergency 1,R0101,Consultation Room
1,Emergency,B0001,Emergency 1,R0102,Consultation Room
2,Emergency,B0001,Emergency 1,R0103,Emergency Room
3,Emergency,B0001,Emergency 1,R0104,Pharmacy
4,Emergency,B0001,Emergency 1,R0201,Laboratory


In [4]:
patient = pd.read_excel("https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/patient.xlsx", engine="openpyxl")
patient.head()

,PatientID,Name,Sex,Contact,email,Address,DOB
0,P0001,Kelsey Moore,female,39472495,kelsey.moore@gmail.com,"34892 Christopher Crossing, South Brooke, NH 1...",1997-05-05
1,P0002,William Perry,male,15351117,william.perry@hotmail.com,"94501 Bell Islands Suite 485, New Jessica, NJ ...",2007-06-06
2,P0003,Nicholas Jones,male,56648257,nicholas.jones@yahoo.com,"8841 Lewis Parkways Apt. 040, South Kristen, I...",1975-12-04
3,P0004,Theresa Smith,female,45807263,theresa.smith@outlook.com,"271 Jeffrey Freeway, Willisbury, ME 39487",1997-09-14
4,P0005,Tara Kennedy,female,84234563,tara.kennedy@protonmail.com,"64128 Rick Coves Apt. 284, Lake James, NJ 51964",2011-02-22


In [5]:
staff = pd.read_csv("https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/staff.csv")
staff.head()

,DoctorID,Name,Contact,Email,DepartmentID,Department
0,E0001,Grace Chen,78901234,grace.chen@abchospital.com,D0005,Pediatrics
1,E0002,Henry Adams,75678901,henry.adams@abchospital.com,D0002,Intensive Care Unit
2,E0003,Maya Patel,44567890,maya.patel@abchospital.com,D0008,Surgery
3,E0004,Sean Underwood,84345678,sean.underwood@abchospital.com,D0001,Emergency
4,E0005,Amy Bryant,72123456,amy.bryant@abchospital.com,D0009,Neurology


# Appointment Data Analysis and Cleaning
---

In [6]:
appointment.head()

,PatientID,DoctorID,BuildingID,RoomID,ScheduledStart,ScheduledEnd,Arrival,ActualStart,ActualEnd,Weight(kg),Temperature(celcius),Comment
0,P0191,E0066,B0001,R0402,2024-01-05 14:00:00,2024-01-05 15:00:00,2024-01-05 14:35:00,2024-01-05 14:35:00,2024-01-05 15:11:00,49.5,37.2,Orientation comfortably
1,P0776,E0065,B0004,R0104,2024-02-27 14:00:00,2024-02-27 15:00:00,2024-02-27 14:59:00,2024-02-27 14:59:00,2024-02-27 16:03:00,55.5,37.1,Imaging has improved slightly
2,P0157,E0051,B0001,R0301,2022-12-28 09:00:00,2022-12-28 10:00:00,NaT,NaT,NaN,NaN,NaN,NaN
3,P0593,E0044,B0004,R0101,2023-01-04 15:00:00,2023-01-04 16:00:00,2023-01-04 15:48:00,2023-01-04 15:48:00,2023-01-04 16:22:00,50.5,36.8,Rash reported today
4,P0434,E0074,B0002,R0203,2020-04-14 09:00:00,2020-04-14 10:00:00,2020-04-14 09:39:00,2020-04-14 09:39:00,2020-04-14 10:07:00,49.5,38.1,Continue observed on forearm


### Checking for nulls


In [7]:
appointment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PatientID             100000 non-null  object        
 1   DoctorID              100000 non-null  object        
 2   BuildingID            100000 non-null  object        
 3   RoomID                100000 non-null  object        
 4   ScheduledStart        100000 non-null  datetime64[ns]
 5   ScheduledEnd          100000 non-null  datetime64[ns]
 6   Arrival               97952 non-null   datetime64[ns]
 7   ActualStart           97952 non-null   datetime64[ns]
 8   ActualEnd             97953 non-null   object        
 9   Weight(kg)            97953 non-null   float64       
 10  Temperature(celcius)  97953 non-null   float64       
 11  Comment               97952 non-null   object        
dtypes: datetime64[ns](4), float64(2), object(6)
memory usage: 

From looking at the non null count, it seems there is a row where it is null for arrival, actualstart, comment but not for actualend, weight and temperature


In [8]:
# Rows where not all arrival, actualstart, actual end, weight, temperature and comment is all null or not null
cols = ['Arrival', 'ActualStart', 'ActualEnd', 'Weight(kg)', 'Temperature(celcius)', 'Comment']

not_null_rows = appointment[appointment[cols].notnull().all(axis=1)]
print("Count of rows where all col is not null:", len(not_null_rows))
selected_rows_null_rows = appointment[appointment[cols].isnull().all(axis=1)]
print("Count of rows where all col is null:", len(selected_rows_null_rows))
# Rows where NOT all are null AND NOT all are not null
mixed_rows = appointment[
    ~(appointment[cols].isnull().all(axis=1)) &
    ~(appointment[cols].notnull().all(axis=1))
]
print("Count of rows where not all col is null and not all col is not null:", len(mixed_rows))
print("Count of rows where all col is null and not all col is not null:", len(selected_rows_null_rows) + len(not_null_rows) + len(mixed_rows))
print("total rows:", len(appointment))
appointment[appointment.index.isin(mixed_rows.index)]

Count of rows where all col is not null: 97952
Count of rows where all col is null: 2048
Count of rows where not all col is null and not all col is not null: 1
Count of rows where all col is null and not all col is not null: 100001
total rows: 100001


,PatientID,DoctorID,BuildingID,RoomID,ScheduledStart,ScheduledEnd,Arrival,ActualStart,ActualEnd,Weight(kg),Temperature(celcius),Comment
100000,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,average,51.497356,37.140657,NaN


From looking at this we can see that there is a row where data is missing, therefore we will remove it.


In [9]:
# remove rows where index is in mixed_rows
appointment = appointment[~appointment.index.isin(mixed_rows.index)]
appointment.info()


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PatientID             100000 non-null  object        
 1   DoctorID              100000 non-null  object        
 2   BuildingID            100000 non-null  object        
 3   RoomID                100000 non-null  object        
 4   ScheduledStart        100000 non-null  datetime64[ns]
 5   ScheduledEnd          100000 non-null  datetime64[ns]
 6   Arrival               97952 non-null   datetime64[ns]
 7   ActualStart           97952 non-null   datetime64[ns]
 8   ActualEnd             97952 non-null   object        
 9   Weight(kg)            97952 non-null   float64       
 10  Temperature(celcius)  97952 non-null   float64       
 11  Comment               97952 non-null   object        
dtypes: datetime64[ns](4), float64(2), object(6)
memory usage: 9.9+ M

#### PatientID, DoctorID, BuildingID, RoomID
---


In [12]:
# Check if PatientID in appointment exists in patient
patient_id_check = appointment['PatientID'].isin(patient['PatientID']).all()
print(f"All PatientIDs in appointment are in patient: {patient_id_check}")

# Check if DoctorID in appointment exists in staff
doctor_id_check = appointment['DoctorID'].isin(staff['DoctorID']).all()
print(f"All DoctorIDs in appointment are in staff: {doctor_id_check}")

# Create a combined key for appointment and facility dataframes
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility[1] + facility[3] # facility column 1 is BuildingID, column 3 is RoomID

# Check if the combined BuildingID and RoomID from appointment exist in facility
building_room_check = appointment['BuildingRoom'].isin(facility['BuildingRoom']).all()
print(f"All BuildingID and RoomID combinations in appointment are in facility: {building_room_check}")

# Drop the temporary combined columns
appointment = appointment.drop(columns=['BuildingRoom'])
facility = facility.drop(columns=['BuildingRoom'])

All PatientIDs in appointment are in patient: True
All DoctorIDs in appointment are in staff: False
All BuildingID and RoomID combinations in appointment are in facility: False


In [22]:
# Find DoctorIDs in staff not in appointment
missing_doctor_ids_in_staff = staff[~staff['DoctorID'].isin(appointment['DoctorID'])]
print("DoctorIDs in staff not in appointment:")
display(missing_doctor_ids_in_staff['DoctorID'].unique())

# Create a combined key for appointment and facility dataframes
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility['BuldingID'] + facility['RoomID'] # Use correct column names

# Find BuildingRoom combinations in facility not in appointment
missing_building_room_in_facility = facility[~facility['BuildingRoom'].isin(appointment['BuildingRoom'])]
print("\nBuildingID and RoomID combinations in facility not in appointment:")
display(missing_building_room_in_facility[['BuildingRoom', 'Department', 'BuldingID', 'BuildingName', 'RoomID', 'Purpose']]) # Display relevant columns from facility using new headers

# Drop the temporary combined columns
appointment = appointment.drop(columns=['BuildingRoom'])
facility = facility.drop(columns=['BuildingRoom'])

DoctorIDs in staff not in appointment:


array(['E0002', 'E0003', 'E0012 ', 'E0013 ', 'E0014 ', 'E0024', 'E0025',
       'E0026', 'E0035', 'E0042', 'E0043', 'E0047', 'E0055', 'E0056',
       'E0061', 'E0062', 'E0071', 'E0077', 'E0078', 'E0082', 'E0086',
       'E0088', 'E0093'], dtype=object)


BuildingID and RoomID combinations in facility not in appointment:


,BuildingRoom,Department,BuldingID,BuildingName,RoomID,Purpose
0,B0001 R0101,Emergency,B0001,Emergency 1,R0101,Consultation Room
1,B0001 R0102,Emergency,B0001,Emergency 1,R0102,Consultation Room
2,B0001 R0103,Emergency,B0001,Emergency 1,R0103,Emergency Room
3,B0001 R0104,Emergency,B0001,Emergency 1,R0104,Pharmacy
4,B0001 R0201,Emergency,B0001,Emergency 1,R0201,Laboratory
5,B0001 R0202,Emergency,B0001,Emergency 1,R0202,Laboratory
6,B0001 R0203,Emergency,B0001,Emergency 1,R0203,X-ray Room
7,B0001 R0204,Emergency,B0001,Emergency 1,R0204,X-ray Room
8,B0001 R0301,Cardiology,B0001,Emergency 1,R0301,Consultation Room
9,B0001 R0302,Pediatrics,B0001,Emergency 1,R0302,Consultation Room


# Facility Data Analysis and Cleaning
---

# Patient Data Analysis and Cleaning
---

# Staff Data Analysis and Cleaning
---